# playing Atari with Deep Learning

In [1]:
# choose game
game = "Pong-v0"

# generate Dataset by random play

* saving only good sequences

In [2]:
import numpy as np
import gym

env = gym.make(game)
x_data = [] # Bilder (States)
y_data = [] # Label (Aktionen)

DATASET_SIZE = 1500
saved_episodes = 0

# Datensatz generieren
the_end = False
while not the_end:
    done = False
    state = env.reset()
    episode_obs = []
    episode_acts = []
  
    while not done:
        action = env.action_space.sample()
        episode_obs.append(state)
        episode_acts.append(action)
        state, reward, done, info = env.step(action)
    
        if reward < 0:
            episode_acts = []
            episode_obs = []
        elif reward > 0:
            x_data += episode_obs
            y_data += episode_acts
            episode_obs = []
            episode_acts = []
            saved_episodes += 1
            if saved_episodes % 100 == 0:
                print(saved_episodes)
            if saved_episodes+1 > DATASET_SIZE:
                print("Dataset completed")
                the_end = True

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
Dataset completed
Dataset completed


# Preprocessing

In [3]:
x_data = np.array(x_data)
y_data = np.array(y_data)
x_data.shape, y_data.shape

((116048, 210, 160, 3), (116048,))

In [4]:
x_data[0].shape

(210, 160, 3)

### resize images to 84x84 pixels

In [5]:
import cv2
def resize(img):
    height = 84
    width = 84
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

In [6]:
resized_x_data = [resize(img) for img in x_data]
resized_x_data = np.array(resized_x_data)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1,2)

axs[0].imshow(x_data[0, :, :, :])
axs[0].set_title("Original", fontsize="15")
axs[1].imshow(resized_x_data[0, :, :, :])
axs[1].set_title("Resized", fontsize="15")
plt.show()

### RGB to Grayscale

In [7]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.reshape(img, (img.shape[0], img.shape[1], 1))
    return img

In [8]:
gray_x_data = [grayscale(img) for img in resized_x_data]
gray_x_data = np.array(gray_x_data)

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(resized_x_data[0, :, :, :])
axs[0].set_title("Resized", fontsize="15")
axs[1].imshow(gray_x_data[0, :, :, 0], cmap="gray")
axs[1].set_title("Grayscale", fontsize="15")
plt.show()

### Frame Stack

In [9]:
from collections import deque
def framestack(dataset):
    data = []
    frames = deque(maxlen=4)

    # initialize first deque
    for _ in range(4):
        frames.append(dataset[0])
    framestack = np.asarray(frames, dtype=np.float32)
    framestack = np.moveaxis(framestack, 0, -1).reshape(84, 84, -1)

    # create deques from dataset
    for state in dataset:
        frames.append(state)
        framestack = np.asarray(frames, dtype=np.float32)
        framestack = np.moveaxis(framestack, 0, -1).reshape(84, 84, -1)
        data.append(framestack)

    return data

In [10]:
stack_data = framestack(gray_x_data) 
stack_data = np.array(stack_data)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(10,5))
axs[0].imshow(stack_data[0+4, :, :, 0], cmap="gray") # +4, because of 4 same initial frames
axs[1].imshow(stack_data[1+4, :, :, 1], cmap="gray")
axs[2].imshow(stack_data[2+4, :, :, 2], cmap="gray")
axs[3].imshow(stack_data[3+4, :, :, 3], cmap="gray")
plt.show()

### converting labels to categorical labels

In [11]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y=y_data, num_classes=env.action_space.n)

### shuffle data

In [12]:
# create random indices
idx = np.random.permutation(len(x_data))
# x - resized grayscale input
# x_stacked - resized grayscale input in framestacks
# labels - actions in to_categorical
x, x_stacked, labels = gray_x_data[idx], stack_data[idx], y[idx]

# create a model


In [13]:
import tensorflow
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

def create_nn(input_shape, output_shape):
    net_input = Input(shape=input_shape)
    x = Conv2D(filters=32, kernel_size=(8, 8), strides=(4, 4), padding="same")(net_input)
    x = Activation("relu")(x)
    x = Conv2D(filters=64, kernel_size=(4, 4), strides=(2, 2), padding="same")(x)
    x = Activation("relu")(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same")(x)
    x = Activation("relu")(x)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation("relu")(x)
    x = Dense(output_shape)(x)
    net_output = Activation("softmax")(x)

    OPTIMIZER = Adam(lr=0.00005)

    model = Model(inputs=net_input, outputs=net_output)
    model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["categorical_accuracy"])

    return model

# Train a model on resized-grayscale input



In [ ]:
x.shape[1:]

In [ ]:
labels.shape[1]

In [ ]:
model_1 = create_nn(x.shape[1:], labels.shape[1])

In [ ]:
EPOCHS = 250

result_1 = model_1.fit(
    x,
    labels,
    batch_size=128,
    epochs=EPOCHS, 
    verbose=1)

In [ ]:
model_1.save_weights("model_1_2.h5")

In [ ]:
for episode in range(5):
    done = False
    episode_reward = 0.0
    state = env.reset()
    state = resize(state)
    state = grayscale(state)
    state = np.reshape(state, (1, 84, 84, 1))
    state = state.astype("float32")
    while not done:
        action = np.argmax(model_1.predict(state))
        state, reward, done, info = env.step(action)
        state = resize(state)
        state = grayscale(state)
        state = np.reshape(state, (1, 84, 84, 1))
        state = state.astype("float32")
        episode_reward += reward
    print("Episode:", episode+1, "\tReward:", episode_reward)

# Train a model on resized-grayscale-framestack input



In [14]:
x_stacked.shape[1:]

(84, 84, 4)

In [15]:
labels.shape[1]

6

In [18]:
model_2 = create_nn(x_stacked.shape[1:], labels.shape[1])

In [19]:
EPOCHS = 85#250

result_2 = model_2.fit(
    x_stacked,
    labels,
    batch_size=128,
    epochs=EPOCHS, 
    verbose=1)

Epoch 1/250
116048/116048 [==============================] - 37s 316us/sample - loss: 2.0898 - categorical_accuracy: 0.1681
Epoch 2/250
116048/116048 [==============================] - 37s 318us/sample - loss: 1.8282 - categorical_accuracy: 0.1736
Epoch 3/250
116048/116048 [==============================] - 36s 307us/sample - loss: 1.8064 - categorical_accuracy: 0.1759
Epoch 4/250
116048/116048 [==============================] - 37s 323us/sample - loss: 1.7969 - categorical_accuracy: 0.1810
Epoch 5/250
116048/116048 [==============================] - 33s 285us/sample - loss: 1.7942 - categorical_accuracy: 0.1806
Epoch 6/250
116048/116048 [==============================] - 37s 318us/sample - loss: 1.7918 - categorical_accuracy: 0.1815
Epoch 7/250
116048/116048 [==============================] - 36s 312us/sample - loss: 1.7896 - categorical_accuracy: 0.1844
Epoch 8/250
116048/116048 [==============================] - 37s 319us/sample - loss: 1.7880 - categorical_accuracy: 0.1858
Epoch 9/

116048/116048 [==============================] - 38s 326us/sample - loss: 0.1502 - categorical_accuracy: 0.9618
Epoch 67/250
116048/116048 [==============================] - 34s 297us/sample - loss: 0.1418 - categorical_accuracy: 0.9636
Epoch 68/250
116048/116048 [==============================] - 36s 314us/sample - loss: 0.1335 - categorical_accuracy: 0.9657
Epoch 69/250
116048/116048 [==============================] - 35s 306us/sample - loss: 0.1275 - categorical_accuracy: 0.9673
Epoch 70/250
116048/116048 [==============================] - 37s 321us/sample - loss: 0.1220 - categorical_accuracy: 0.9691
Epoch 71/250
116048/116048 [==============================] - 36s 312us/sample - loss: 0.1154 - categorical_accuracy: 0.9711
Epoch 72/250
116048/116048 [==============================] - 33s 288us/sample - loss: 0.1100 - categorical_accuracy: 0.9716
Epoch 73/250
116048/116048 [==============================] - 37s 317us/sample - loss: 0.1070 - categorical_accuracy: 0.9719
Epoch 74/250


KeyboardInterrupt: 

In [20]:
model_2.save_weights("model_2.h5")

In [31]:
for episode in range(5):
    done = False
    episode_reward = 0.0
    state = env.reset()
    state = resize(state)
    state = grayscale(state)
    state = np.reshape(state, (84, 84, 1))
    frames = deque(maxlen=4)
    # initialize deque for framestack
    for _ in range(4):
        frames.append(state)
    framestack = np.asarray(frames, dtype=np.float32)
    framestack = np.moveaxis(framestack, 0, -1).reshape(1, 84, 84, 4)#reshape(84, 84, -1)

    # playing a episode
    while not done:
        action = np.argmax(model_2.predict(framestack))
        state, reward, done, info = env.step(action)
        episode_reward += reward
        state = resize(state)
        state = grayscale(state)
        state = np.reshape(state, (84, 84, 1))
        frames.append(state)
        framestack = np.asarray(frames, dtype=np.float32)
        framestack = np.moveaxis(framestack, 0, -1).reshape(1, 84, 84, 4)
        
    print("Episode:", episode+1, "\tReward:", episode_reward)

Episode: 1 	Reward: -18.0
Episode: 2 	Reward: -18.0
Episode: 3 	Reward: -19.0
Episode: 4 	Reward: -16.0
Episode: 5 	Reward: -16.0
